# Perform a model using only Redis and files. 
Here we will load the trace and predicted values obtained from the HEC (or third party computer infrastructure).
Trace (in data frame ) and predicted values [clean] (in dataframe)
using *PyMC3*

In [1]:
# The cannonical import
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()

#import traversals.strategies as st
#from os import walk
import matplotlib.pyplot as plt
import pandas as pd
import pymc3 as pm
import redis
## Use the ggplot style
plt.style.use('ggplot')

/opt/conda/envs/biospytial/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Reading from a CSV file (imported from trace to dataframe(PYMC3)

In [12]:
## Reading from a CSV file (imported from trace to dataframe(PYMC3)
path = "/mnt/data1/outputs/presence_only_models/polygono4-guerrero/resultados/test-polygono4/"

smadvi = "tracetestmodel-guerrero4-advi.csv"
smc_500 = "tracetestmodel-guerrero4-5its.csv"
smc_2500 = "tracetestmodel-guerrero4.csv"
madvi = pd.read_csv(path+smadvi,encoding='utf8')
mc_500 = pd.read_csv(path+mc_500,encoding='utf8')
mc_2500 = pd.read_csv(path+mc_2500,encoding='utf8')

In [14]:
mc_500

,Unnamed: 0,tau,phi,b__0,b__1,b__2,b__3,b__4,b__5,b__6,...,latent_field__494,latent_field__495,latent_field__496,latent_field__497,latent_field__498,latent_field__499,latent_field__500,latent_field__501,latent_field__502,sigma
0,0,0.293337,0.018099,-0.467533,0.025734,-0.011593,-0.039557,0.010458,0.153284,0.003563,...,1.114529,1.105484,1.185199,1.077467,1.148027,1.187569,1.099515,1.130926,1.095158,0.000339
1,1,21.849675,0.010069,2.113162,0.019589,0.022741,-0.017019,0.025219,-0.088814,0.149254,...,1.114727,1.101099,1.150897,1.132570,1.057066,1.026724,1.125252,1.092984,1.132073,0.000320
2,2,0.214860,0.008389,-0.235725,-0.023644,0.081424,0.015584,0.000976,-0.131176,0.106579,...,1.110248,1.096544,1.129761,1.138019,1.108849,1.143853,1.107249,1.094103,1.085366,0.000330
3,3,2.420084,0.004605,0.920682,-0.025962,0.020504,0.005189,0.008754,-0.129812,0.155522,...,1.117031,1.062623,1.124932,1.151429,1.126399,1.141093,1.106660,1.104099,1.096914,0.000291
4,4,2.647068,0.006222,0.416268,-0.047796,0.036556,-0.032247,-0.006105,-0.033031,0.211898,...,1.136679,1.107041,1.106675,1.149331,1.099635,1.172357,1.109550,1.107202,1.073903,0.000325
5,5,4.842395,0.009647,0.752861,-0.012054,-0.083976,-0.008940,-0.005270,0.138193,-0.007252,...,1.149673,1.082392,1.115752,1.102248,1.178298,1.125234,1.154479,1.094292,1.138598,0.000325
6,6,10.510715,0.019606,-0.324801,0.024075,0.062635,-0.011290,0.073123,-0.016309,0.032705,...,1.148971,1.090254,1.129464,1.098550,1.095687,1.084286,1.158282,1.120179,1.143157,0.000319
7,7,5.026072,0.006418,0.267864,-0.002623,0.013715,-0.059042,0.035057,0.000345,0.097554,...,1.125619,1.099535,1.132505,1.154197,1.171199,1.163363,1.110312,1.118104,1.122010,0.000263
8,8,2.768215,0.013919,-1.661339,0.058472,0.047853,0.069302,0.084313,0.132834,0.324592,...,1.148198,1.089716,1.149550,1.138406,1.146607,1.108363,1.139152,1.117035,1.158207,0.000232
9,9,1.065181,0.016948,-1.322189,0.040813,0.049838,0.048594,0.064586,0.035784,0.077901,...,1.136989,1.116999,1.120115,1.139056,1.194632,1.127491,1.105573,1.116746,1.177497,0.000230


In [3]:
conn = redis.StrictRedis(host='redis',password='biospytial.')
subs = conn.pubsub()
## Subscribe to the datachannel
subs.subscribe('datachannel')

In [12]:
## Get data
data_message = subs.get_message()
if data_message:
    print("got message")
else:
    print("nothing yet")

nothing yet


In [7]:
data_message

{'channel': 'datachannel', 'data': 1L, 'pattern': None, 'type': 'subscribe'}

In [ ]:
## This is for calculating the signal
from pymc3.variational.callbacks import CheckParametersConvergence
def FitMyModel(trainDM,PredDM):
    with pm.Model() as model:
        # partition dataframes df
        Ydf = trainDM[0]
        TXdf = trainDM[1]     
        PXdf = PredDM  
        ## Parameters for linear predictor
        #b0 = pm.Normal('b0',mu=0,sd=10)
        #dum_names = filter(lambda col : str(col).startswith('inegiv5name'),TXdf)
        #dumsdf = TXdf[dum_names]
        #dumshape = dumscols.shape
        #coordsdf = TXdf[['Longitude','Latitude']] 
        # Create vectors for dumi vars 
        #drvs = map(lambda col : pm.Normal(col,mu=0,sd=1.5),dum_names)
        ## Create theano vector
        dimX = len(TXdf.columns)
        b = pm.Normal('b',mu=0,sd=1.5,shape=dimX)
        #mk = pm.math.matrix_dot(TXdf.values,b.transpose())
        ## The latent function
        x_index = TXdf.columns.get_loc("Longitude")
        y_index = TXdf.columns.get_loc("Latitude")
        ## Building the covariance structure
        tau = pm.HalfNormal('tau',sd=10)
        sigma = pm.HalfNormal('sigma',sd=10)
        #phi = pm.Uniform('phi',0,15)
        phi = pm.HalfNormal('phi',sd=6)
        Tau = pm.gp.cov.Constant(tau)
        cov = (sigma * pm.gp.cov.Matern32(2,phi,active_dims=[x_index,y_index])) + Tau
        mean_f = pm.gp.mean.Linear(coeffs=b)
        gp = pm.gp.Latent(mean_func=mean_f,cov_func=cov)
        f = gp.prior("latent_field", X=TXdf.values,reparameterize=False)
        yy = pm.Bernoulli("yy",logit_p=f,observed=Ydf.values)
        #trace = pm.fit(method='advi', callbacks=[CheckParametersConvergence()],n=15000)    
        trace = pm.sample(15,init='adapt_diag')
        #trace = trace.sample(draws=5000)
        # Remove any column that doesnt appear in the training data
        ValidPreds = PredDM[TXdf.columns]
        PredX = ValidPreds.values
        f_star = gp.conditional("f_star", PredX)
        pred_samples = pm.sample_ppc(trace, vars=[f_star], samples=100)
        return pred_samples,trace
 